In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from scipy import sparse
import scipy
import sys
sys.path.append('..')
from importlib import reload
import optim as opm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [68]:
def loss_1(A):
#     A = -A**2
    A = torch.nn.ReLU()(-A)
    return - torch.trace(A) + A.sum()
    
    
class unitary_solver(torch.nn.Module):
    def __init__(self,sps_list):
        super(unitary_solver, self).__init__()
        self.sps_list = sps_list
        self.generators = []
        self._n_params = []
        for N in sps_list:
            self.generators.append(torch.tensor(self.make_generator(N), requires_grad=False))
            self._n_params.append(int(N*(N-1)/2))
        tmp = torch.rand(np.sum(self._n_params))
        self._params = torch.nn.Parameter(tmp)
        self._size = np.prod(sps_list)
        
    def forward(self, x):
        assert x.shape[0] == x.shape[1], "require square matrix"
        assert x.shape[0] == self._size
        x = self.matrix @ x @ self.matrix.T
        return x
    
    
    @property
    def params(self):
        return self._params
    
    @property
    def n_params(self):
        return self._n_params
    
    def set_params(self,params):
        self._params[:] = params[:]
    
    @property
    def matrix(self):
        tmp = self._params[:,None,None] * self.generators[i]
        return torch.matrix_exp(tmp.sum(axis=0))
    
    @staticmethod
    def make_generator(N):
        tmp_list = []
        for i, j in [[i,j] for i in range(N) for j in range(i+1,N)]:
            tmp = np.zeros((N,N), dtype=np.float64)
            tmp[i,j] = 1
            tmp[j,i] = -1
            tmp_list.append(tmp)
        return np.array(tmp_list)

In [69]:
model(X)

tensor([[ 0.7625, -1.6185,  0.6985, -0.7313],
        [-1.6185,  0.9239, -0.8712,  0.3864],
        [ 0.6985, -0.8712,  0.1769, -1.6185],
        [-0.7313,  0.3864, -1.6185,  0.6398]], dtype=torch.float64,
       grad_fn=<MmBackward0>)

In [70]:
loss_1(y)

tensor(9.6791, dtype=torch.float64, grad_fn=<AddBackward0>)

In [71]:
X = torch.rand((4,4),dtype=torch.float64)
X = X + X.T
model = unitary_solver([2,2])
optimizer = torch.optim.SGD(model.parameters(), lr=0.03, momentum=0, weight_decay=0)

In [72]:
for t in range(2000):
    y = model(X)
    loss = loss_1(y)
    if t % 1000 == 0:
#         print(y)
        print(t, loss.item())
        print(next(model.parameters()))
        print("\n")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 1.6556026824586438
Parameter containing:
tensor([0.7011, 0.6087], requires_grad=True)


1000 0.0
Parameter containing:
tensor([0.2785, 0.2788], requires_grad=True)




In [73]:
model.matrix @ X @ model.matrix.T

tensor([[2.1223, 1.1205, 1.2999, 0.1002],
        [1.1205, 1.0407, 0.4026, 0.0177],
        [1.2999, 0.4026, 1.2854, 0.1551],
        [0.1002, 0.0177, 0.1551, 0.3699]], dtype=torch.float64,
       grad_fn=<MmBackward0>)